In [1]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  
site = 'cstec_org_cn_publication'
site_name = 'China Science and Technology Exchange Center (China)'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_fetch_cat_url = 0
publish_source = 'cstec.org.cn'
country = 'China'
language = 'English'
images_path = []
foot_fall = c.get_foot_fall(publish_source)

headers = {
                'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
        }

#http://cstec.org.cn/recentnews.html?column=1027
cat_urls = ('http://www.cstec.org.cn/handlers/cistcMenuInfoList.ashx?columnid=1021&isall=1&keyword=&year=&pagenum=1',
            'http://www.cstec.org.cn/handlers/cistcInfoPicPageList.ashx?columnid=1020&topN=6&pagenum=1&contentlength=300')
for cat_url in cat_urls:
    
    logger.info(f'Fetching cat url: {cat_url}\n')
    url = requests.get(cat_url, headers=headers).json()  # to take it in json format
    
    page = url['infolist']
    print(page)
    break

    for i in page:
        source_link_id = i["InfoId"]
        r = i["InfoUrl"]
        source_column = re.sub('\S+column=','',r,re.S)

        # source_link
        source_link = f'http://www.cstec.org.cn/handlers/cistcInfo.ashx?infoid={source_link_id}&contentLenth=&column={source_column}'
    
       # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue 
            
        logger.info(f'Fetching {source_link}\n')
        time.sleep(random.randint(1,3))

        page = requests.get(source_link, headers=headers).json()
          
        # source_headline
        source_headline = page["InfoTitle"]
        
        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue
        
        # Date and time
        pub_date, publish_time = '', ''

        try:
            date_time_str = page["InfoPublicDate"] + '000000'
            date_time_obj = datetime.strptime(date_time_str, '%Y/%m/%d%H%M%S')
            ist_date_time = date_time_obj + timedelta(hours = 0,minutes = 0)  
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]
        except:
            pass
        
        # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')
            skipped_due_to_date += 1
            continue
        # break if date is not today's date
#         if pub_date != created_on:
#             break

        # journalist   
        journalist = ''
        if not journalist: journalist = 'NA'        
        
        # source_content          
        source_content = page["InfoContent"]
        source_content = c.strip_html(source_content)
        
        # skip if content not found
        if not source_content:
            logger.info(f'Skipping due to content {source_link}\n')
            skipped_due_to_content += 1
            continue
      
        # current date and time 
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []

        # creating dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":foot_fall,
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content
                    } 

        cl_data.insert_one(clientdata)  
        no_of_data += 1   
    
logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch cat url: {unable_to_fetch_cat_url}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'country : {country}\n')
logger.info(f'language : {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')


[{'MenuIsPublish': 'True', 'MenuIsDeleted': 'False', 'Infotheme': None, 'InfoId': 96808, 'InfoMenu': 0, 'InfoTitle': 'Sino-Irish Cleantech Digital Roadshow and Industry Connect Successfully Held', 'InfoViceTitle': None, 'InfoUrl': 'infoDetail.html?id=96808&column=1021', 'InfoContent': None, 'InfoPublicDate': '2022/6/2', 'InfoProvider': None, 'InfoPerson': None, 'InfoPicture': None, 'InfoPictures': None}, {'MenuIsPublish': 'True', 'MenuIsDeleted': 'False', 'Infotheme': None, 'InfoId': 96795, 'InfoMenu': 0, 'InfoTitle': 'CSTEC inks strategic cooperation framework agreement with the Government of Mentougou District of Beijing', 'InfoViceTitle': None, 'InfoUrl': 'infoDetail.html?id=96795&column=1021', 'InfoContent': None, 'InfoPublicDate': '2022/5/26', 'InfoProvider': None, 'InfoPerson': None, 'InfoPicture': None, 'InfoPictures': None}, {'MenuIsPublish': 'True', 'MenuIsDeleted': 'False', 'Infotheme': None, 'InfoId': 96794, 'InfoMenu': 0, 'InfoTitle': 'CSTEC follows up on China-Netherlands 

In [1]:
import requests

json_file = requests.get('http://www.cstec.org.cn/handlers/cistcMenuInfoList.ashx?columnid=1021&isall=1&keyword=&year=&pagenum=1').text

In [12]:
import json
file = json.loads(json_file)# to convert string to dictionary

In [42]:
file['Pagenum']

'1'

'1'